In [2]:
import torch
from torchvision import transforms, models
from PIL import Image
import os

MODEL_PATH = 'best_fruit_model.pt'
IMAGE_PATH = 'apple.jpeg'  
IMAGE_SIZE = 224
CLASS_NAMES = ['Apple', 'Banana', 'Grapes', 'Guava', 'Mango', 'Orange', 'Papaya', 'Pomegranate', 'Strawberry', 'Watermelon']  

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


model = models.resnet18(pretrained=False)
model.fc = torch.nn.Sequential(
    torch.nn.Linear(model.fc.in_features, 256),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.3),
    torch.nn.Linear(256, len(CLASS_NAMES))
)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.to(device)
model.eval()

image = Image.open(IMAGE_PATH).convert('RGB')
input_tensor = transform(image).unsqueeze(0).to(device)

with torch.no_grad():
    output = model(input_tensor)
    predicted_idx = output.argmax(1).item()
    predicted_class = CLASS_NAMES[predicted_idx]

print(f"Predicted Fruit: {predicted_class}")

Predicted Fruit: Apple
